In [1]:
import os 
import time 
from u_mlinterp import mlinterps,mlinterpe
from u_riointerp import riointerp
from upaths import OUT_TILES_DPATH
import pandas as pd
from glob import glob
from concurrent.futures import ProcessPoolExecutor

In [2]:
from upaths import OT_DEMs_PATTERN,GEDI_L3_TPATH,gedi_dsm_global_fn,gedi_dtm_global_fn
from rutils import get_raster_info, gdal_regrid
import numpy as np 
import math 
import rasterio 

x in meters: 12.0
Grid size in degrees for 30m: 0.0002777777777777778


In [3]:
def demvfill_unspervised(dem_ipath,dem_opath,method='riointerp',si=0):
    ti = time.perf_counter()
    print(os.path.isfile(dem_ipath))
    if method == 'mlinterps':
        print(f'using {method}')
        dem_opath = dem_opath.replace('.tif', f'_{method}.tif')
        if not os.path.isfile(dem_opath):
            mlinterps(dem_ipath, dem_opath, model_type='catboost')
        else:
            print(f'file aready created\nsaved at {dem_opath}')

    elif method == 'mlinterpe':
        print(f'using {method}')
        dem_opath = dem_opath.replace('.tif', f'_{method}.tif')
        if not os.path.isfile(dem_opath):
            mlinterpe(dem_ipath, dem_opath, model_type='catboost')
        else:
            print(f'file aready created\nsaved at {dem_opath}')

    elif method == 'riointerp':
        print(f'using {method}')
        dem_opath = dem_opath.replace('.tif', f'_{method}.tif')
        if not os.path.isfile(dem_opath):
            riointerp(dem_ipath, dem_opath, smoothing_iterations=si)
        else:
            print(f'file aready created\nsaved at {dem_opath}')

    else:
        print('method not available: try mlinterps,mlinterpe,riointerp')
    tf = time.perf_counter() -ti 
    print(f'run.time ={tf/60} min(s)')
    print(f'method @{method}')

In [4]:
name = 'GEDI_L3'
files = glob(OT_DEMs_PATTERN)
files = [i for i in files if name in i]
files = list(set(files))
print(len(files))
tile_dpath = GEDI_L3_TPATH

17


In [ ]:
for fi in files:
    tilename = fi.split('/')[-3]
    dout_path = os.path.join(tile_dpath,tilename)
    os.makedirs(dout_path, exist_ok=True)
    chm_tile = os.path.join(dout_path, f'{tilename}_{os.path.basename(gedi_dsm_global_fn)}')
    dtm_tile = os.path.join(dout_path, f'{tilename}_{os.path.basename(gedi_dtm_global_fn)}')
    proj, xres, yres, xmin, xmax, ymin, ymax, w, h = get_raster_info(fi)
    #gdal_regrid(fi, chm_tile, xmin, ymin, xmax, ymax, xres, yres,mode="num", t_epsg='EPSG:4979', overwrite=False)
    #gdal_regrid(fi, dtm_tile, xmin, ymin, xmax, ymax, xres, yres,mode="num", t_epsg='EPSG:4979', overwrite=False)

    dtm_tile_fil = dtm_tile#.replace('.tif', '_riofill.tif')
    chm_tile_fil = chm_tile#.replace('.tif', '_riofill.tif')
    # try:
    #     os.remove(dtm_tile_fil)
    #     os.remove(chm_tile_fil)
    # except:
    #     continue

    demvfill_unspervised(chm_tile,chm_tile_fil,method='riointerp',si=0)
    demvfill_unspervised(dtm_tile,dtm_tile_fil,method='riointerp',si=0)
